In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook creates the reporting units dataset. It does not, however, summarize data into the reporting units.

The geoprocessing workflow for creating these reporting units is quite complex, but the overall goal is to have a more or less uniform set of polygons that maintain parcel boundary lines, while combining small parcels, and subdividing large parcels, while also paying attention to the WUI interface and intermix zones.

There is also an outputted APN table which can be joined to these reporting units to determine which APNs correspond to which reporting unit. 

**Input Data Layers:**
- Expanded WUI/AOI (created in `1_Expanded_WUI.ipynb`)
- [SBC Parcels](https://databasin.org/datasets/008da0ba3dc14be18aa3bcfd90dc9615/)
- [SBC Boundary](https://databasin.org/datasets/1cdc07adea7d4dee9cc1f07ab44cfef3/)
- [WUI Intermix & Interface](https://databasin.org/datasets/218e54cf56f94342a46fefcf05c64f9a/)

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

**Import starting data layers and create WUI merge**

In [3]:
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')

in_data = ["SBC_Parcels", "SBC_Boundary", "WUI_Intermix", "WUI_Interface"]

for sf in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)

path = os.path.join(THIS_FOLDER, "out_data/expanded_wui/Expanded_WUI.shp")
arcpy.CopyFeatures_management(path, "Expanded_WUI")

arcpy.Merge_management(["WUI_Interface_rpj", "WUI_Intermix_rpj"], 
                       "WUI_Merge")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\WUI_Merge'>

**Create the Roads and Gaps layer**

In [4]:
arcpy.Clip_analysis("SBC_Parcels_rpj", "SBC_Boundary_rpj", 
                    "SBC_Parcels_NoIslands")
arcpy.analysis.Erase("SBC_Boundary_rpj", "SBC_Parcels_NoIslands", 
                     "Roads_Gaps", None)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Roads_Gaps'>

**Clip the parcels and roads and gaps to the expanded WUI geometry**

In [5]:
arcpy.Clip_analysis("SBC_Parcels_NoIslands", "Expanded_WUI", "WUI_Parcels_Clip")
arcpy.Clip_analysis("Roads_Gaps", "Expanded_WUI", "Roads_Gaps_Clip")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Roads_Gaps_Clip'>

**Add parcels that are less than or equal to 150 acres and greater than or equal to 10 acres to the final reporting units layer**

In [6]:
acre = 43560

In [7]:
selection = arcpy.management.SelectLayerByAttribute("WUI_Parcels_Clip", "NEW_SELECTION", "Shape_Area <= {} And Shape_Area >= {}".format(150 * acre, 10 * acre), None)
arcpy.CopyFeatures_management(selection, "LTE_Lg_GTE_Sm")
reporting_units = ["LTE_Lg_GTE_Sm"]

**Select parcels that are larger than 150 acres and subdivide them into 100 acre subdivisions, then add that to the reporting units layer**

In [8]:
selection = arcpy.management.SelectLayerByAttribute("WUI_Parcels_Clip", "NEW_SELECTION", "Shape_Area > {}".format(150 * acre), None)
arcpy.management.SubdividePolygon(selection, "GT150_SubDiv100", "EQUAL_AREAS", None, "{} SquareFeet".format(100 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT150_SubDiv100")

**Select parcels that are less than 10 acres and dissolve boundaries (this makes the city blocks individual polygons)**

In [9]:
selection = arcpy.management.SelectLayerByAttribute("WUI_Parcels_Clip", "NEW_SELECTION", "Shape_Area < {}".format(10 * acre), None)
arcpy.gapro.DissolveBoundaries(selection, "LT10_Dis", "SINGLE_PART", None, None, None)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\LT10_Dis'>

**From the newly dissolved boundaries layer, select polygons that are larger that 150 acres and subdivide those into 100 acres subdivisions and add those to the reporting units**

In [10]:
selection = arcpy.management.SelectLayerByAttribute("LT10_Dis", "NEW_SELECTION", "Shape_Area > {}".format(150 * acre), None)
arcpy.management.SubdividePolygon(selection, "GT_Lg_Dis_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(100 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT_Lg_Dis_SubDiv")

**Again from the dissolved boundaries layer, select parcels that are less than or equal to 150 acres and add those to the reporting units**

In [11]:
selection = arcpy.management.SelectLayerByAttribute("LT10_Dis", "NEW_SELECTION", "Shape_Area <= {}".format(150 * acre), None)
arcpy.CopyFeatures_management(selection, "LTE_Lg")
reporting_units.append("LTE_Lg")

**Now subdivide the WUI-clipped Roads and Gaps layer into 10 acre subdivisions and add that to the reporting units**

In [12]:
arcpy.management.SubdividePolygon("Roads_Gaps_Clip", "Roads_Gaps_Clip_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(10 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("Roads_Gaps_Clip_SubDiv")

**Merge all the various reporting units and turn them into singlepart polygons. We need to do this in order to elimate sliver polygons**

In [13]:
arcpy.Merge_management(reporting_units, "Merge1")
arcpy.management.MultipartToSinglepart("Merge1", "Merge1_SP")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Merge1_SP'>

**Eliminate polygons less than 10 acres (need to do it twice, not sure why)**

In [14]:
selection = arcpy.management.SelectLayerByAttribute("Merge1_SP", "NEW_SELECTION", "Shape_Area < {}".format(10 * acre), None)
arcpy.management.Eliminate(selection, "Merge1_SP_Elim", "LENGTH", "", None)

selection = arcpy.management.SelectLayerByAttribute("Merge1_SP_Elim", "NEW_SELECTION", "Shape_Area < {}".format(2 * acre), None)
arcpy.management.Eliminate(selection, "Merge1_SP_Elim_2", "LENGTH", "", None)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Merge1_SP_Elim_2'>

**Now we handle the parcels and polygons that are in the WUI intermix/interface areas. These polygons have smaller subdivision rules (10 acre subdivisions). We also delete unneccessary fields**

In [15]:
selection = arcpy.SelectLayerByLocation_management("Merge1_SP_Elim_2", "INTERSECT", "WUI_Merge")
arcpy.CopyFeatures_management(selection, "Priority_Parcels")
arcpy.management.SubdividePolygon("Priority_Parcels", "Priority_Parcels_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(10 * acre), None, 0, "STACKED_BLOCKS")
reporting_units = ["Priority_Parcels_SubDiv"]

selection = arcpy.SelectLayerByLocation_management("Merge1_SP_Elim_2", "INTERSECT", "WUI_Merge", invert_spatial_relationship = "INVERT")
arcpy.CopyFeatures_management(selection, "Non_Priority_Parcels")
reporting_units.append("Non_Priority_Parcels")

arcpy.Merge_management(reporting_units, "Merge_Reporting_Units")
arcpy.management.MultipartToSinglepart("Merge_Reporting_Units", "Merge_Reporting_Units_SP")

FCfields = [f.name for f in arcpy.ListFields("Merge_Reporting_Units_SP")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape']
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Merge_Reporting_Units_SP", fields2Delete)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Merge_Reporting_Units_SP'>

**Here we create the APN table that corresponds to the reporting units**

In [16]:
arcpy.analysis.SummarizeWithin("Merge_Reporting_Units_SP", "WUI_Parcels_Clip", "Reporting_Units", "KEEP_ALL", None, "ADD_SHAPE_SUM", "SQUAREKILOMETERS", "apn", "NO_MIN_MAJ", "NO_PERCENT", "apn_Summary")

FCfields = [f.name for f in arcpy.ListFields("Reporting_Units")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape', "Join_ID"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Reporting_Units", fields2Delete)

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable("Reporting_Units")
fieldmappings.addTable("WUI_Parcels_Clip")

keepers = ["Join_ID, apn"]

pcFieldIndex = fieldmappings.findFieldMapIndex("apn")
pcfieldmap = fieldmappings.getFieldMap(pcFieldIndex)
pcnew = pcfieldmap.outputField
pcnew.name = "apn_join"
pcnew.aliasName = "apn_join"
pcfieldmap.outputField = pcnew
pcfieldmap.mergeRule = "First"

fieldmappings.replaceFieldMap(pcFieldIndex, pcfieldmap)

for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))

arcpy.SpatialJoin_analysis("Reporting_Units", "WUI_Parcels_Clip", 
                           "Reporting_Units_APN", "JOIN_ONE_TO_ONE", "#", fieldmappings) 

FCfields = [f.name for f in arcpy.ListFields("Reporting_Units_APN")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape', "Join_ID", "apn"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Reporting_Units_APN", fields2Delete)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Reporting_Units.gdb\\Reporting_Units_APN'>

**Finally export the Expanded WUI intersecting parcels, the reporting units, and the APN table**

In [17]:
dir_path = os.path.join(THIS_FOLDER, "out_data/ru")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

xcel_path = os.path.join(THIS_FOLDER, "APN_RU_Table.xlsx")
arcpy.conversion.TableToExcel("apn_Summary", xcel_path, "NAME", "CODE")

arcpy.conversion.FeatureClassToShapefile("WUI_Parcels_Clip", 
                                        os.path.join(THIS_FOLDER, "out_data/ru"))
arcpy.conversion.FeatureClassToShapefile("Reporting_Units_APN", 
                                        os.path.join(THIS_FOLDER, "out_data/ru"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\ru'>